![image](dcdw.png)

In [31]:
import random, math
import numpy as np

def sigmoid(x):
    return 1/(1+np.exp(-x))

def swish(x):
    return x*sigmoid(x)

def relu(x):
    return np.maximum(0,x)

def activation_function(z,act):
    if act=="sigmoid":
        return sigmoid(z)
    elif act == "swish":
        return swish(z)
    elif act == "relu":
        return relu(z)
    elif act=="tanh":
        return np.tanh(z)
    
class Layer():
    def __init__(self,input_size,output_size,act=None):
        self.size = input_size
        self.output_size = output_size
        self.activation = act
        self.weights = np.random.uniform(-1,1,(self.output_size,self.size))    
        self.bias = np.random.uniform(-1,1,(output_size,1))
        self.A = np.random.uniform(-1,1,(output_size,1))
        self.Z = np.random.uniform(-1,1,(output_size,1))
        
    def forward_propagation(self, input_data):
        self.Z = np.dot(self.weights,input_data)+self.bias
        self.A = activation_function(self.Z, self.activation)
        
    def derMSE(self, target):
        return 2*(self.A - target)
    
    def descent(self, input_data, gradient, learningRate):
        
        if(self.activation == "tanh"):
            derZ = 1 - np.power(self.A, 2)
        elif(self.activation == "swish"):
            derZ = swish(self.Z) + sigmoid(self.Z) * (1 - swish(self.Z))
        reps = (self.weights.shape[0], 1)
        derWeights = np.tile(input_data.transpose(), reps)        # Size -> a(L-1)*a(L)
        """derWeights is a matrix with derivatives of Z WRT weights, which is transposed inputs, 
        repeated in rows n-times, where n is number of neurons.
        
    Example:
    
        input_data = [
            [2],
            [1],
            [0]
        ]
        
        derWeights = [
            [2, 1, 0],
            [2, 1, 0],
            [2, 1, 0],
            ... n-rows
        ]
        
        """

        
        """ The below part adds the gradient to the derivative of A WRT input_data and passes this new
        gradient through return, to be used as the gradient for next layer's descent.
            dA/dX is made with 2 steps: adding the backprop gradient to the derivative of A WRT Z and then adding
        the derivative of Z WRT input_data (chain rule).
        
        1.
        Since A is a matrix shaped Nx1, where N is the number of outputs, the receiving gradient from the upper layer
        must be the same shape. Therefore we can multiply the gradient and the derivative together element-wise.
        
    Example:
        
         derZ = [3x1]
         gradient = [3x1]
         firstGrad = [3x1] *(elementwise) [3x1] = [3x1]
        
        """
        firstGrad = np.multiply(gradient, derZ)             # ∂C/∂a(L) * ∂(act)/∂Z    a(L)*1 * a(L)*1 = a(L)*1
        
        """
        2.
        What's left is adding the gradient of Z WRT input_data. 
        
        This turns out to be the weights matrix. Now we have to multiply the firstGrad gradient to these weights 
        element-wise but since the firstGrad is Nx1 shape and the weights are NxM, where M are the features, 
        we need to reshape the gradient matrix to match the weights matrix by cloning gradient's columns:
        """
        secondGrad = np.tile(firstGrad, (1, self.weights.shape[1]))          #Size -> a(L)*1 -> a(L)*a(L-1)
        
        """
        Finally we multiply (E-W) secondGrad to the weights matrix:
        """
        derX = np.multiply(self.weights,secondGrad)         #Size -> a(L)*a(L-1) * a(L)*a(L-1) = a(L)*a(L-1)
        
        """But because same inputs are multiplied with many weights, we can sum those weights together. It turns out
        that we can sum columns to do that"""
        
        derX = np.sum(derX, axis=0, keepdims=True)       #Size -> 1*a(L-1) 
        derBias = 1
        
        weightGrad = np.multiply(derWeights, np.tile(firstGrad, (1, self.weights.shape[1])))
        self.weights = self.weights - learningRate * weightGrad
        self.bias = self.bias - learningRate * firstGrad
        
        """We return transposed matrix, because we desire inputs with a shape of Nx1 and right now finalGrad is 
        transposed"""
        return derX.transpose()     # Size -> a(L-1)*1
    
        
class NeuralNetwork():
    def __init__(self):
        self.layers=[]
        self.epochs=10
        self.learning_rate = 0.008
    
    def add_layer(self,input_size,output_size,activation=None):
        new_layer = Layer(input_size,output_size,activation)
        self.layers.append(new_layer)
        
    def forward_propagation(self,layer_no):
        current_layer = self.layers[layer_no-1]
        prev_layer = self.layers[layer_no-2]
        act = current_layer.activation
        input_data = prev_layer.A
        self.Z = np.dot(weights,input_data)+self.bias
        result = activation_function(self.Z,act)    # array containing neuron values
        current_layer.A = result            #After forward propogation, fills in the neurons in that layer
        return result
    
    def full_forward_propagation(self, input_data):
        #print("layer 0 forward_propagation")
        
        self.layers[0].forward_propagation(input_data)        # From input data to first layer
        for i in range(1, len(self.layers)):
            #print("layer " + str(i) + " forward_propagation")
            
            self.layers[i].forward_propagation(self.layers[i-1].A)      #From layer i-1 to layer i 
        return self.layers[len(self.layers)-1].A
    
    def back_propagation(self, input_data, target):
        gradient = self.layers[len(self.layers)-1].derMSE(target)     # ∂C/∂a(L)   Size -> a(L)*1
        for i in range(0, len(self.layers)-1):
            index = len(self.layers)-1 - i
            #print("Layer " + str(index) + " backpropagation")
            gradient = self.layers[index].descent(self.layers[index-1].A, gradient, self.learning_rate)   #a(L)*1
        self.layers[0].descent(input_data, gradient, self.learning_rate)
            
            
    def predict(self,test_data):
        self.layers[0].forward_propagation(test_data)
        for i in range(1, len(self.layers)):
            self.layers[i].forward_propagation(self.layers[i-1].A)
        return self.layers[len(self.layers)-1].A

train_data = np.array([[[3],[2],[1]],
                      [[5],[1],[1]],
                      [[5],[0],[0]],
                      [[3],[0],[1]],
                      [[4],[-1],[2]],
                      [[3],[-1],[5]],
                      [[3],[-1],[11]],
                      [[3],[1],[3]],
                      [[-1],[1],[1]],
                      [[2],[0],[1]],
                      [[0],[0],[1]],
                      [[3],[0],[5]],
                      [[2],[-1],[6]],
                      [[1],[1],[-4]],
                      [[0],[0],[-1]],
                      [[2],[1],[-4]],
                      [[2],[1],[-1]],
                      [[1],[0],[-1]],
                      [[10],[4],[-2]]])

"""Here we generate target_data set using formula 2*x0 + -4*x1 + -x2"""
target_data = []
for i in range(0, len(train_data)):
    if train_data[i][0]*2 + train_data[i][1]*-4 + train_data[i][2]*1 > 0:
        target_data.append([[1], [0]])
    else:
        target_data.append([[0], [1]])

network = NeuralNetwork()
network.add_layer(3, 1, "tanh")
network.add_layer(1, 2, "tanh")
network.add_layer(2, 2, "swish")

for i in range(0, 800):
    for j in range(0,len(train_data)):
        print("----------------")
        print("Input_data:\n" + str(train_data[j]))
        print("Forward pass:\n" + str(network.full_forward_propagation(train_data[j])))
        print("Target Data:\n" + str(target_data[j]))
        network.back_propagation(train_data[j], target_data[j])
print("Prediction : ", network.predict([0,0,2]))



----------------
Input_data:
[[3]
 [2]
 [1]]
Forward pass:
[[-0.06408639]
 [ 0.17953778]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[5]
 [1]
 [1]]
Forward pass:
[[-0.0674114 ]
 [ 0.18657033]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[5]
 [0]
 [0]]
Forward pass:
[[-0.05836903]
 [ 0.18383458]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [0]
 [1]]
Forward pass:
[[-0.0485818 ]
 [ 0.18159001]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 4]
 [-1]
 [ 2]]
Forward pass:
[[-0.03907919]
 [ 0.17865714]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 3]
 [-1]
 [ 5]]
Forward pass:
[[-0.0284927 ]
 [ 0.17661435]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 3]
 [-1]
 [11]]
Forward pass:
[[-0.01413573]
 [ 0.17882522]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [1]
 [3]]
Forward pass:
[[-0.00538986]
 [ 0.17393479]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[-1]
 [ 1]
 [ 1]]
Forward pass:
[[0.16736908]
 [0

[[0], [1]]
----------------
Input_data:
[[ 1]
 [ 0]
 [-1]]
Forward pass:
[[0.65582108]
 [0.32088301]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[10]
 [ 4]
 [-2]]
Forward pass:
[[0.67167275]
 [0.3113078 ]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [2]
 [1]]
Forward pass:
[[0.67631284]
 [0.31053252]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[5]
 [1]
 [1]]
Forward pass:
[[0.65967874]
 [0.31874639]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[5]
 [0]
 [0]]
Forward pass:
[[0.67152893]
 [0.31233538]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [0]
 [1]]
Forward pass:
[[0.68242962]
 [0.30659187]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 4]
 [-1]
 [ 2]]
Forward pass:
[[0.69410029]
 [0.30020911]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 3]
 [-1]
 [ 5]]
Forward pass:
[[0.70432843]
 [0.29483912]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 3]
 [-1]
 [11]]
Forward pass:
[[0.71010461]
 [0.2

----------------
Input_data:
[[5]
 [1]
 [1]]
Forward pass:
[[0.66781563]
 [0.31674111]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[5]
 [0]
 [0]]
Forward pass:
[[0.67942383]
 [0.31033865]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [0]
 [1]]
Forward pass:
[[0.68987372]
 [0.30464381]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 4]
 [-1]
 [ 2]]
Forward pass:
[[0.70145791]
 [0.29824005]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 3]
 [-1]
 [ 5]]
Forward pass:
[[0.71144567]
 [0.29279137]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 3]
 [-1]
 [11]]
Forward pass:
[[0.71845386]
 [0.28913403]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [1]
 [3]]
Forward pass:
[[0.72703824]
 [0.2845007 ]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[-1]
 [ 1]
 [ 1]]
Forward pass:
[[0.47661888]
 [0.52152748]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[2]
 [0]
 [1]]
Forward pass:
[[0.72998272]
 [0.28424739]]
Targ

 [0.29480262]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 2]
 [ 1]
 [-4]]
Forward pass:
[[0.72170195]
 [0.28666627]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 2]
 [ 1]
 [-1]]
Forward pass:
[[0.6911329 ]
 [0.30327347]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 1]
 [ 0]
 [-1]]
Forward pass:
[[0.66777917]
 [0.31661551]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[10]
 [ 4]
 [-2]]
Forward pass:
[[0.68915832]
 [0.30455016]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [2]
 [1]]
Forward pass:
[[0.68327056]
 [0.30829761]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[5]
 [1]
 [1]]
Forward pass:
[[0.67628546]
 [0.31219491]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[5]
 [0]
 [0]]
Forward pass:
[[0.6877108 ]
 [0.30586866]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [0]
 [1]]
Forward pass:
[[0.69773792]
 [0.30033496]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 4]
 [-1]
 [ 2]]
Forwa

 [0.26062234]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 2]
 [-1]
 [ 6]]
Forward pass:
[[0.78341514]
 [0.25561245]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 1]
 [ 1]
 [-4]]
Forward pass:
[[0.70757452]
 [0.29876622]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 0]
 [ 0]
 [-1]]
Forward pass:
[[0.69467174]
 [0.30497185]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 2]
 [ 1]
 [-4]]
Forward pass:
[[0.71852025]
 [0.28981227]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 2]
 [ 1]
 [-1]]
Forward pass:
[[0.69335408]
 [0.30335556]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 1]
 [ 0]
 [-1]]
Forward pass:
[[0.67513873]
 [0.31361318]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[10]
 [ 4]
 [-2]]
Forward pass:
[[0.70573242]
 [0.29665804]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [2]
 [1]]
Forward pass:
[[0.68402306]
 [0.30891316]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[5]
 [1]
 [1]]

...

Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [1]
 [3]]
Forward pass:
[[ 1.0204518 ]
 [-0.01135703]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[-1]
 [ 1]
 [ 1]]
Forward pass:
[[-0.0898234 ]
 [ 1.02712161]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[2]
 [0]
 [1]]
Forward pass:
[[ 1.01958003]
 [-0.01125638]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[0]
 [0]
 [1]]
Forward pass:
[[0.88949118]
 [0.05243795]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [0]
 [5]]
Forward pass:
[[ 1.02390727]
 [-0.01192028]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 2]
 [-1]
 [ 6]]
Forward pass:
[[ 1.02283081]
 [-0.01179891]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 1]
 [ 1]
 [-4]]
Forward pass:
[[-0.0896784 ]
 [ 1.02643455]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 0]
 [ 0]
 [-1]]
Forward pass:
[[0.10644896]
 [0.95564411]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 2]
 [ 1]
 [-4]]
Forward 

Input_data:
[[3]
 [1]
 [3]]
Forward pass:
[[ 1.02029624]
 [-0.01117567]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[-1]
 [ 1]
 [ 1]]
Forward pass:
[[-0.08908354]
 [ 1.02635598]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[2]
 [0]
 [1]]
Forward pass:
[[ 1.01943085]
 [-0.01107645]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[0]
 [0]
 [1]]
Forward pass:
[[0.89069746]
 [0.05136715]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [0]
 [5]]
Forward pass:
[[ 1.02370152]
 [-0.01172139]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 2]
 [-1]
 [ 6]]
Forward pass:
[[ 1.02263397]
 [-0.0116019 ]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 1]
 [ 1]
 [-4]]
Forward pass:
[[-0.08892879]
 [ 1.02568883]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 0]
 [ 0]
 [-1]]
Forward pass:
[[0.1056219 ]
 [0.95672999]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 2]
 [ 1]
 [-4]]
Forward pass:
[[-0.08649716]
 [ 1.02615285]]
Targ

----------------
Input_data:
[[0]
 [0]
 [1]]
Forward pass:
[[0.89178169]
 [0.05040957]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [0]
 [5]]
Forward pass:
[[ 1.02351584]
 [-0.01154211]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 2]
 [-1]
 [ 6]]
Forward pass:
[[ 1.02245633]
 [-0.01142433]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 1]
 [ 1]
 [-4]]
Forward pass:
[[-0.0882405 ]
 [ 1.02502726]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 0]
 [ 0]
 [-1]]
Forward pass:
[[0.10486868]
 [0.95769304]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 2]
 [ 1]
 [-4]]
Forward pass:
[[-0.08585954]
 [ 1.02550827]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 2]
 [ 1]
 [-1]]
Forward pass:
[[0.07665749]
 [0.97696427]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 1]
 [ 0]
 [-1]]
Forward pass:
[[0.87263514]
 [0.06428327]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[10]
 [ 4]
 [-2]]
Forward pass:
[[0.94803506]
 [

----------------
Input_data:
[[0]
 [0]
 [1]]
Forward pass:
[[0.89283923]
 [0.04948032]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [0]
 [5]]
Forward pass:
[[ 1.02333402]
 [-0.01136685]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 2]
 [-1]
 [ 6]]
Forward pass:
[[ 1.0222824 ]
 [-0.01125074]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 1]
 [ 1]
 [-4]]
Forward pass:
[[-0.08755599]
 [ 1.02439037]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 0]
 [ 0]
 [-1]]
Forward pass:
[[0.10412481]
 [0.95861999]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 2]
 [ 1]
 [-4]]
Forward pass:
[[-0.08522396]
 [ 1.02488703]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 2]
 [ 1]
 [-1]]
Forward pass:
[[0.07601603]
 [0.97773169]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 1]
 [ 0]
 [-1]]
Forward pass:
[[0.87367995]
 [0.06325895]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[10]
 [ 4]
 [-2]]
Forward pass:
[[0.94859398]
 [

----------------
Input_data:
[[0]
 [0]
 [1]]
Forward pass:
[[0.89387136]
 [0.04857824]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[3]
 [0]
 [5]]
Forward pass:
[[ 1.02315588]
 [-0.01119548]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 2]
 [-1]
 [ 6]]
Forward pass:
[[ 1.02211201]
 [-0.01108101]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[ 1]
 [ 1]
 [-4]]
Forward pass:
[[-0.08687549]
 [ 1.02377717]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 0]
 [ 0]
 [-1]]
Forward pass:
[[0.10338993]
 [0.95951233]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 2]
 [ 1]
 [-4]]
Forward pass:
[[-0.08459073]
 [ 1.02428828]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 2]
 [ 1]
 [-1]]
Forward pass:
[[0.07538539]
 [0.97846863]]
Target Data:
[[0], [1]]
----------------
Input_data:
[[ 1]
 [ 0]
 [-1]]
Forward pass:
[[0.87470269]
 [0.06226075]]
Target Data:
[[1], [0]]
----------------
Input_data:
[[10]
 [ 4]
 [-2]]
Forward pass:
[[0.94914015]
 [